# XGBoost
Construir e otimizar modelos com gradient boosting ("ensemble method", como RandomForest, que combina previsões de diversos modelos (várias árvores, no caso da floresta))

## Gradient Boosting
Passa por ciclos e iterativamente adiciona modelos ao conjunto<br>
Inicializa o conjunto com apenas um modelo, que pode ter previsões simples<br>
- Ciclo:
    - Usa conjunto atual pra gerar previsões pra cada observação no dataset; adiciona as previsões de todos modelos no conjunto
    - Usa as previsões para calcular uma função de erro (como MAE)
    - Usa função de erro pra treinar um novo modelo que vai ser adicionado ao conjunto. Determina parâmetros do modelo para que adicionado esse modelo reduza o erro (usa-se "gradient descent" na função de erro pra achar os novos parametros)
    - Adiciona o novo modelo ao conjunto
    - Repete o ciclo

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv("melb_data.csv")

# Select subset of predictors
cols_to_use = ["Rooms", "Distance", "Landsize", "BuildingArea", "YearBuilt"]
X = data[cols_to_use]

# Select target
y = data.Price

# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [4]:
from xgboost import XGBRegressor  # extreme gradient boosting

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [5]:
from sklearn.metrics import mean_absolute_error

pred = my_model.predict(X_valid)
print("MAE - ", mean_absolute_error(pred, y_valid))

MAE -  239206.26977172313


### Parametros
- n_estimators: quantas vezes vai fazer o ciclo (número de modelos que vai adicionar no conjunto)
    - se poucos, tem underfitting e, com muitos, overfitting
    - 100-1000, dependendo do learning_rate
- early_stopping_rounds: automaticamente encontrar o valor ideal para n_estimators; para quando o score de validação para de aumentar
    - pode colocar valor alto para n_estimators e colocar o early_stop
    - o valor dado é o mínimo de ciclos que vai fazer antes de começar a poder parar automaticamente
    - se usado, precisa colocar dados para calcular a validção (eval_set)
- learning_rate: multiplicar as previsões para cada modelo por um número pequeno antes de adicionar; cada árvore ajuda menos, então pode colocar valor de n_estimators maior sem ter overfitting
    - geralmente baixo learning_rate com alto n_estimators dá mais precisão, ainda que leve mais tempo porque vai ter mais ciclos
- n_jobs: em datasets mairoes, pode usar paralelismo para tornar os modelos mais rápidos; em datasets menores não vai ajudar muito
    - pode colocar o número de cores na CPU

In [17]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.033, n_jobs=4, early_stopping_rounds=5)
my_model.fit(X_train, y_train,
            #  early_stopping_rounds=5,
             eval_set=[(X_valid, y_valid)],
             verbose=False)

pred = my_model.predict(X_valid)
print("MAE - ", mean_absolute_error(pred, y_valid))

MAE -  237782.10690353462
